 # TTM zero-shot and few-shot benchmarking on multiple datasets

 **Using TTM-1024-96-v1 model.**

## Imports

In [1]:
import math

import matplotlib.pyplot as plt
import pandas as pd
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed

from tsfm_public import TinyTimeMixerForPrediction, TrackingCallback, count_parameters, load_dataset
from tsfm_public.toolkit.visualization import plot_predictions

## Important arguments

In [2]:
# Set seed
set_seed(42)

# Specify model parameters
context_length = 1024
forecast_length = 96
freeze_backbone = True
learning_rate = 0.001

# Other args
EPOCHS = 50
NUM_WORKERS = 16

# Make sure all the datasets in the following `list_datasets` are
# saved in the `DATA_ROOT_PATH` folder. Or, change it accordingly.
# Refer to the load_dataset() function
# in notebooks/hfdemo/tinytimemixer/utils/ttm_utils.py
# to see how it is used.
DATA_ROOT_PATH = "datasets/"

# This is where results will be saved
OUT_DIR = "ttm_results_benchmark_1024_96_tmp"

## List of benchmark datasets (TTM was not pre-trained on any of these)

In [3]:
list_datasets = [
    "etth1",
    "etth2",
    "ettm1",
    "ettm2",
    "weather",
    "electricity",
    "traffic",
]

## Get model path

In [4]:
hf_model_path = "ibm/TTM"
if context_length == 512:
    hf_model_branch = "main"
elif context_length == 1024:
    hf_model_branch = "1024_96_v1"
else:
    raise ValueError("Current supported context lengths are 512 and 1024. Stay tuned for more TTMs!")

## Main benchmarking loop

In [5]:
all_results = {
    "dataset": [],
    "zs_mse": [],
    "fs5_mse": [],
    "fs10_mse": [],
    "zs_eval_time": [],
    "fs5_mean_epoch_time": [],
    "fs5_total_train_time": [],
    "fs10_mean_epoch_time": [],
    "fs10_total_train_time": [],
    "fs5_best_val_metric": [],
    "fs10_best_val_metric": [],
}
# Loop over data
for DATASET in list_datasets:
    print()
    print("=" * 100)
    print(
        f"Running zero-shot/few-shot for TTM-{context_length} on dataset = {DATASET}, forecast_len = {forecast_length}"
    )
    print(f"Model will be loaded from {hf_model_path}/{hf_model_branch}")
    SUBDIR = f"{OUT_DIR}/{DATASET}"

    # Set batch size
    if DATASET == "traffic":
        BATCH_SIZE = 8
    elif DATASET == "electricity":
        BATCH_SIZE = 32
    else:
        BATCH_SIZE = 64

    # Data prep: Get dataset
    _, _, dset_test = load_dataset(DATASET, context_length, forecast_length, dataset_root_path=DATA_ROOT_PATH)

    #############################################################
    ##### Use the pretrained model in zero-shot forecasting #####
    #############################################################
    # Load model
    zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(hf_model_path, revision=hf_model_branch)

    # zeroshot_trainer
    zeroshot_trainer = Trainer(
        model=zeroshot_model,
        args=TrainingArguments(
            output_dir=f"{SUBDIR}/zeroshot",
            per_device_eval_batch_size=BATCH_SIZE,
        ),
        eval_dataset=dset_test,
    )

    # evaluate = zero-shot performance
    print("+" * 20, "Test MSE zero-shot", "+" * 20)
    zeroshot_output = zeroshot_trainer.evaluate(dset_test)
    print(zeroshot_output)
    print("+" * 60)
    all_results["zs_eval_time"].append(zeroshot_output["eval_runtime"])

    # Plot
    plot_predictions(
        model=zeroshot_trainer.model,
        dset=dset_test,
        plot_dir=SUBDIR,
        num_plots=10,
        plot_prefix="test_zeroshot",
        channel=0,
    )
    plt.close()

    # write results
    all_results["dataset"].append(DATASET)
    all_results["zs_mse"].append(zeroshot_output["eval_loss"])

    ################################################################
    ## Use the pretrained model in few-shot 5% and 10% forecasting #
    ################################################################
    for fewshot_percent in [5, 10]:
        print("-" * 20, f"Running few-shot {fewshot_percent}%", "-" * 20)
        # Data prep: Get dataset
        dset_train, dset_val, dset_test = load_dataset(
            DATASET,
            context_length,
            forecast_length,
            fewshot_fraction=fewshot_percent / 100,
            dataset_root_path=DATA_ROOT_PATH,
        )

        # change head dropout to 0.7 for ett datasets
        if "ett" in DATASET:
            finetune_forecast_model = TinyTimeMixerForPrediction.from_pretrained(
                hf_model_path, revision=hf_model_branch, head_dropout=0.7
            )
        else:
            finetune_forecast_model = TinyTimeMixerForPrediction.from_pretrained(
                hf_model_path, revision=hf_model_branch
            )

        if freeze_backbone:
            print(
                "Number of params before freezing backbone",
                count_parameters(finetune_forecast_model),
            )

            # Freeze the backbone of the model
            for param in finetune_forecast_model.backbone.parameters():
                param.requires_grad = False

            # Count params
            print(
                "Number of params after freezing the backbone",
                count_parameters(finetune_forecast_model),
            )

        print(f"Using learning rate = {learning_rate}")
        finetune_forecast_args = TrainingArguments(
            output_dir=f"{SUBDIR}/fewshot_{fewshot_percent}",
            overwrite_output_dir=True,
            learning_rate=learning_rate,
            num_train_epochs=EPOCHS,
            do_eval=True,
            evaluation_strategy="epoch",
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            dataloader_num_workers=NUM_WORKERS,
            report_to=None,
            save_strategy="epoch",
            logging_strategy="epoch",
            save_total_limit=1,
            logging_dir=f"{SUBDIR}/fewshot_{fewshot_percent}",  # Make sure to specify a logging directory
            load_best_model_at_end=True,  # Load the best model when training ends
            metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
            greater_is_better=False,  # For loss
        )

        # Create the early stopping callback
        early_stopping_callback = EarlyStoppingCallback(
            early_stopping_patience=10,  # Number of epochs with no improvement after which to stop
            early_stopping_threshold=0.0,  # Minimum improvement required to consider as improvement
        )
        tracking_callback = TrackingCallback()

        # Optimizer and scheduler
        optimizer = AdamW(finetune_forecast_model.parameters(), lr=learning_rate)
        scheduler = OneCycleLR(
            optimizer,
            learning_rate,
            epochs=EPOCHS,
            steps_per_epoch=math.ceil(len(dset_train) / (BATCH_SIZE)),
        )

        finetune_forecast_trainer = Trainer(
            model=finetune_forecast_model,
            args=finetune_forecast_args,
            train_dataset=dset_train,
            eval_dataset=dset_val,
            callbacks=[early_stopping_callback, tracking_callback],
            optimizers=(optimizer, scheduler),
        )

        # Fine tune
        finetune_forecast_trainer.train()

        # Evaluation
        print(
            "+" * 20,
            f"Test MSE after few-shot {fewshot_percent}% fine-tuning",
            "+" * 20,
        )
        fewshot_output = finetune_forecast_trainer.evaluate(dset_test)
        print(fewshot_output)
        print("+" * 60)

        # Plot
        plot_predictions(
            model=finetune_forecast_trainer.model,
            dset=dset_test,
            plot_dir=SUBDIR,
            num_plots=10,
            plot_prefix=f"test_fewshot_{fewshot_percent}",
            channel=0,
        )
        plt.close()

        # write results
        all_results[f"fs{fewshot_percent}_mse"].append(fewshot_output["eval_loss"])
        all_results[f"fs{fewshot_percent}_mean_epoch_time"].append(tracking_callback.mean_epoch_time)
        all_results[f"fs{fewshot_percent}_total_train_time"].append(tracking_callback.total_train_time)
        all_results[f"fs{fewshot_percent}_best_val_metric"].append(tracking_callback.best_eval_metric)

    df_out = pd.DataFrame(all_results).round(3)
    print(df_out[["dataset", "zs_mse", "fs5_mse", "fs10_mse"]])
    df_out.to_csv(f"{OUT_DIR}/results_zero_few.csv")
    df_out.to_csv(f"{OUT_DIR}/results_zero_few.csv")

INFO:p-3361:t-22942224311040:data_handling.py:load_dataset:Dataset name: etth1, context length: 1024, prediction length 96
INFO:p-3361:t-22942224311040:data_handling.py:load_dataset:Data lengths: train = 7521, val = 2785, test = 2785



Running zero-shot/few-shot for TTM-1024 on dataset = etth1, forecast_len = 96
Model will be loaded from ibm/TTM/1024_96_v1


++++++++++++++++++++ Test MSE zero-shot ++++++++++++++++++++


{'eval_loss': 0.3621068000793457, 'eval_model_preparation_time': 0.0026, 'eval_runtime': 1.3961, 'eval_samples_per_second': 1994.831, 'eval_steps_per_second': 31.516}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


INFO:p-3361:t-22942224311040:data_handling.py:load_dataset:Dataset name: etth1, context length: 1024, prediction length 96
INFO:p-3361:t-22942224311040:data_handling.py:load_dataset:Data lengths: train = 285, val = 2785, test = 2785


...

/u/wmgiffor/git/tsfm/tsfm_public/toolkit/dataset.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df["group"] = 0  # create a artificial group
/u/wmgiffor/git/tsfm/tsfm_public/toolkit/dataset.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df["group"] = 0  # create a artificial group
/u/wmgiffor/git/tsfm/tsfm_public/toolkit/dataset.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

Number of params before freezing backbone 946336
Number of params after freezing the backbone 389984
Using learning rate = 0.001


Epoch,Training Loss,Validation Loss
1,0.270300,0.362664
2,0.250300,0.351346
3,0.241200,0.348280
4,0.235000,0.348637
5,0.230200,0.346396
6,0.226200,0.343058
7,0.223100,0.351453
8,0.220700,0.347172
9,0.218700,0.349513
10,0.216700,0.351570


[TrackingCallback] Mean Epoch Time = 14.365050464868546 seconds, Total Train Time = 488.8848283290863
++++++++++++++++++++ Test MSE after few-shot 10% fine-tuning ++++++++++++++++++++


{'eval_loss': 0.41844481229782104, 'eval_runtime': 31.8365, 'eval_samples_per_second': 107.204, 'eval_steps_per_second': 13.412, 'epoch': 16.0}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
       dataset  zs_mse  fs5_mse  fs10_mse
0        etth1   0.362    0.361     0.363
1        etth2   0.281    0.280     0.280
2        ettm1   0.387    0.372     0.371
3        ettm2   0.175    0.173     0.172
4      weather   0.152    0.151     0.150
5  electricity   0.156    0.145     0.138
6      traffic   0.458    0.416     0.418


## Benchmarking results

In [6]:
df_out

,dataset,zs_mse,fs5_mse,fs10_mse,zs_eval_time,fs5_mean_epoch_time,fs5_total_train_time,fs10_mean_epoch_time,fs10_total_train_time,fs5_best_val_metric,fs10_best_val_metric
0,etth1,0.362,0.361,0.363,1.396,0.645,26.784,0.665,19.867,0.658,0.663
1,etth2,0.281,0.280,0.280,0.764,0.611,16.502,0.695,19.077,0.224,0.223
2,ettm1,0.387,0.372,0.371,2.973,0.960,40.540,1.342,42.534,0.409,0.408
3,ettm2,0.175,0.173,0.172,3.098,0.976,30.684,1.297,33.768,0.121,0.121
4,weather,0.152,0.151,0.150,5.272,1.386,37.157,2.053,44.430,0.419,0.424
5,electricity,0.156,0.145,0.138,25.149,4.614,703.978,8.607,795.040,0.112,0.109
6,traffic,0.458,0.416,0.418,47.419,7.176,653.298,14.365,488.885,0.345,0.343


## Comparison with recent pre-trained model

The following tables compare the TTM-1024-96 model's performance with the recently released [MOIRAI model](https://arxiv.org/abs/2402.02592).

### Comparing forecasting error (MSE)

![TTM_vs_MOIRAI](images/ttm_moirai.png)

### Comparing model size (in Millions)

| Model | Number of parameters | MOIRAI size over TTM | 
|:-----:|:--------------------:|:--------------------:|
| TTM-1024-96-v1 | 0.9 M | - |
| MOIRAI_Small | 14 M | **16X** |
| MOIRAI_Base | 91 M | **101X** |
| MOIRAI_Large | 311 M | **346X** |

## Conclusion

TTM stands out as a pioneering set of compact pre-trained models for time series forecasting. As demonstrated in the preceding results, TTM exhibits a remarkable ability to outperform the recently released MOIRAI model by a substantial margin. Additionally, TTM is significantly more compact, ranging from 16 to 346 times smaller than MOIRAI, showcasing its efficiency and effectiveness in comparison.